In [5]:
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.hyundai.com/kr/ko/purchase-event/vehicles-review")
driver.maximize_window()
time.sleep(3)

car_reviews = []

for page in range(1, 51):
    print(f"\n📄 현재 페이지: {page}")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    review_list = driver.find_elements(By.CSS_SELECTOR, "#reviewList li")

    for idx, review_item in enumerate(review_list):
        try:
            data_id = review_item.get_attribute("data-id")
            li_class = review_item.get_attribute("class") or ""
            li_style = review_item.get_attribute("style") or ""

            if "clone" in li_class or "display:none" in li_style:
                continue

            if len(review_item.find_elements(By.CSS_SELECTOR, ".title-wrap .title")) == 0:
                continue

            driver.execute_script("arguments[0].scrollIntoView(true);", review_item)
            time.sleep(0.3)

            # 자동차 이름 (span.best 제거)
            title_element = review_item.find_element(By.CSS_SELECTOR, ".title-wrap .title")
            try:
                span_best = title_element.find_element(By.TAG_NAME, "span")
                driver.execute_script("arguments[0].remove();", span_best)
            except:
                pass
            car_title = title_element.text.strip()

            # 더보기 버튼 있는지 + 텍스트 확인
            has_more_btn = False
            try:
                toggle_btn = review_item.find_element(By.CLASS_NAME, "toggle-btn")
                toggle_text = toggle_btn.text.strip()
                if toggle_text == "더보기":
                    has_more_btn = True
                    driver.execute_script("arguments[0].click();", toggle_btn)
                    time.sleep(0.5)
            except:
                pass

            # 리뷰 내용 가져오기
            review_html = ""
            try:
                if has_more_btn:
                    review_elem = review_item.find_element(By.CSS_SELECTOR, ".desc .review-text .text.show-more")
                else:
                    review_elem = review_item.find_element(By.CSS_SELECTOR, ".desc .review-text .text")
                review_html = review_elem.get_attribute("innerHTML")
            except:
                review_html = ""

            review_text = review_html.replace("<br>", "\n").strip()

            # 태그 정보 가져오기 (딕셔너리로)
            tags_dict = {}
            try:
                category_items = review_item.find_elements(By.CSS_SELECTOR, ".category-list .list li")
                for cat in category_items:
                    tag_name = cat.find_element(By.CSS_SELECTOR, ".flag").text.strip()
                    tag_desc = cat.find_element(By.CSS_SELECTOR, ".txt").text.strip()
                    tags_dict[tag_name] = tag_desc
            except:
                pass  # 태그 없을 수도 있음

            # 데이터 저장
            car_reviews.append({
                "data_id": data_id,
                "car_name": car_title,
                "review": review_text,
                "tags": tags_dict
            })

            print(f"✅ {idx + 1}: [{data_id}] {car_title} - {review_text[:30]}... | tags: {list(tags_dict.keys())}")

        except Exception as e:
            continue

    if page < 50:
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "button.navi.next")
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(3)
        except Exception as e:
            print(f"❌ 페이지 {page + 1} 이동 실패: {e}")
            break

driver.quit()

with open("hyundai_car_reviews.json", "w", encoding="utf-8") as f:
    json.dump(car_reviews, f, ensure_ascii=False, indent=2)

print("\n✅ 최종 저장 완료: hyundai_car_reviews.json")


📄 현재 페이지: 1
✅ 1: [5167] 더 뉴 투싼 Hybrid - 구매를 하기전 타사 차량과 고민을 많이 했지만  선루프... | tags: ['성능', '공간', '디자인', '승차감']
✅ 11: [5156] 코나 Hybrid - 신차는 첫 차로 구매했습니다. 

1.승차감 
기존에는... | tags: ['성능', '공간', '디자인', '승차감']
✅ 17: [5114] 디 올 뉴 팰리세이드 - 기존 투싼과의 9년 동행을 마무리하며 새로운 차량을 고... | tags: ['성능', '공간', '디자인', '승차감']
✅ 24: [5113] 더 뉴 아이오닉 5 - 10년 넘게 경유차를 몰다가 처음 전기차로 갈아탔습니다... | tags: ['성능', '공간', '디자인', '승차감']
✅ 34: [5088] 그랜저 Hybrid - ( 새로운 출발을 함께하는 내 '브리즈'와의 첫 만남)... | tags: ['성능', '공간', '디자인', '승차감']
✅ 40: [5005] 더 뉴 투싼 Hybrid - 2025 투싼 하이브리드 진짜 만족하면서 타고 있어요.... | tags: ['성능', '공간', '디자인', '승차감']
✅ 46: [4962] 싼타페 - 나의 두 번째 자동차, 싼타페와의 설레는 동행
나의 첫... | tags: ['성능', '공간', '디자인', '승차감']
✅ 54: [4899] 더 뉴 아이오닉 5 - 오래오래 탈 차로 여러가지 차량을 알아보다가 아이오닉5... | tags: ['성능', '공간', '디자인', '승차감']
✅ 63: [4951] 싼타페 - 이제 두 달 뒤면 세 식구가 됩니다.
아직 세상에 나오... | tags: ['성능', '공간', '디자인', '승차감']
✅ 70: [4529] 아이오닉9 - 기존에 운행하고 있던 차량은 렉서스NX300h 하이브리... | tags: ['성능', '공간', '디자인', '승차감']

📄 현재 페이지: 2
✅ 1: [9061] 코나 - 5월말 계약 6월말 출고 빠른출고 설명